In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.stats import skew, kurtosis, entropy
from scipy.signal import welch, hilbert
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Publication-quality settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 12
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 11
plt.rcParams['ytick.labelsize'] = 11
plt.rcParams['legend.fontsize'] = 11

# ====================================================================================
# CONFIGURATION
# ====================================================================================
base_path = r"E:\Upwork Project\AI_Leak_Detection_Project\data\raw\Accelerometer\Looped"
leak_classes = ['Circumferential Crack', 'Gasket Leak', 'Longitudinal Crack', 'No-leak', 'Orifice Leak']
samples_to_keep = 200
samples_to_plot_per_class = 3
output_dir = 'manuscript_figures'
os.makedirs(output_dir, exist_ok=True)

# Color scheme for each class
class_colors = {
    'Circumferential Crack': '#E63946',
    'Gasket Leak': '#F77F00',
    'Longitudinal Crack': '#06A77D',
    'No-leak': '#457B9D',
    'Orifice Leak': '#A23B72'
}

# Sampling frequency (adjust if different)
fs = 1000  # Hz

# ====================================================================================
# DATA LOADING AND PREPROCESSING
# ====================================================================================
print("="*80)
print("LOADING AND PREPROCESSING DATA")
print("="*80)

all_data = []
all_labels = []
plot_data = {cls: [] for cls in leak_classes}
raw_stats = {cls: [] for cls in leak_classes}

for leak_type in leak_classes:
    class_path = os.path.join(base_path, leak_type)
    files = [f for f in os.listdir(class_path) if f.endswith('.csv')]
    
    print(f"\n📁 Processing {leak_type}: {len(files)} files")
    
    for idx, file in enumerate(files):
        file_path = os.path.join(class_path, file)
        try:
            df = pd.read_csv(file_path)
            signal_raw = df['Value'].values
            
            # Store raw statistics
            raw_stats[leak_type].append({
                'filename': file,
                'length': len(signal_raw),
                'mean': np.mean(signal_raw),
                'std': np.std(signal_raw),
                'min': np.min(signal_raw),
                'max': np.max(signal_raw),
                'rms': np.sqrt(np.mean(signal_raw**2)),
                'peak_to_peak': np.ptp(signal_raw)
            })
            
            # Normalize signal
            signal_norm = (signal_raw - np.mean(signal_raw)) / (np.std(signal_raw) + 1e-8)
            
            # Truncate or pad
            if len(signal_norm) > samples_to_keep:
                signal_norm = signal_norm[:samples_to_keep]
            else:
                signal_norm = np.pad(signal_norm, (0, samples_to_keep - len(signal_norm)))
            
            all_data.append(signal_norm)
            all_labels.append(leak_type)
            
            # Save samples for plotting
            if len(plot_data[leak_type]) < samples_to_plot_per_class:
                plot_data[leak_type].append({
                    'signal': signal_norm,
                    'raw': signal_raw,
                    'filename': file
                })
                
        except Exception as e:
            print(f"  ❌ Error reading {file}: {e}")

# Convert to arrays
X = np.array(all_data)
y = np.array(all_labels)

print(f"\n{'='*80}")
print(f"✅ Total processed samples: {X.shape[0]}")
print(f"✅ Signal length: {X.shape[1]}")
print(f"✅ Classes: {leak_classes}")
print(f"{'='*80}\n")

# Save preprocessed data
np.save(f'{output_dir}/X_accelerometer_looped.npy', X)
np.save(f'{output_dir}/y_accelerometer_looped.npy', y)

# ====================================================================================
# PLOT 1: SAMPLE COUNT DISTRIBUTION (BAR CHART)
# ====================================================================================
print("Creating Plot 1: Sample Count Distribution...")

fig, ax = plt.subplots(figsize=(12, 6))
class_counts = pd.Series(y).value_counts()
colors = [class_colors[cls] for cls in leak_classes]

bars = ax.bar(leak_classes, [class_counts[cls] for cls in leak_classes], 
              color=colors, edgecolor='black', linewidth=2, alpha=0.8)

for i, (cls, count) in enumerate(zip(leak_classes, [class_counts[cls] for cls in leak_classes])):
    ax.text(i, count + 2, str(count), ha='center', va='bottom', 
            fontweight='bold', fontsize=13)

ax.set_ylabel('Number of Samples', fontweight='bold', fontsize=14)
ax.set_xlabel('Leak Type', fontweight='bold', fontsize=14)
ax.set_title('Dataset Distribution Across Leak Classes', fontsize=16, fontweight='bold', pad=20)
ax.set_xticklabels(leak_classes, rotation=30, ha='right')
ax.grid(axis='y', alpha=0.3, linestyle='--', linewidth=1)
ax.set_axisbelow(True)

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_01_Sample_Distribution.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_01_Sample_Distribution.png")

# ====================================================================================
# PLOT 2: TIME-DOMAIN SIGNALS (OVERLAPPED)
# ====================================================================================
print("Creating Plot 2: Time-Domain Signals (Overlapped)...")

fig, ax = plt.subplots(figsize=(14, 7))

for leak_type in leak_classes:
    for sample in plot_data[leak_type]:
        time_axis = np.arange(len(sample['signal'])) / fs
        ax.plot(time_axis, sample['signal'], 
                color=class_colors[leak_type], alpha=0.4, linewidth=1.5)

# Create legend with one entry per class
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], color=class_colors[cls], lw=3, label=cls) 
                   for cls in leak_classes]
ax.legend(handles=legend_elements, loc='upper right', framealpha=0.9, fontsize=11)

ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)
ax.set_ylabel('Normalized Amplitude', fontweight='bold', fontsize=14)
ax.set_title('Time-Domain Signals: All Leak Classes Comparison', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim([0, samples_to_keep/fs])

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_02_Time_Domain_Overlapped.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_02_Time_Domain_Overlapped.png")

# ====================================================================================
# PLOT 3: TIME-DOMAIN SIGNALS (STACKED SUBPLOTS)
# ====================================================================================
print("Creating Plot 3: Time-Domain Signals (Stacked)...")

fig, axes = plt.subplots(5, 1, figsize=(14, 14))

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    for sample in plot_data[leak_type]:
        time_axis = np.arange(len(sample['signal'])) / fs
        ax.plot(time_axis, sample['signal'], 
                color=class_colors[leak_type], alpha=0.6, linewidth=1.2)
    
    ax.set_ylabel('Normalized\nAmplitude', fontweight='bold', fontsize=12)
    ax.set_title(f'{leak_type}', fontsize=13, fontweight='bold', 
                loc='left', color=class_colors[leak_type], pad=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim([0, samples_to_keep/fs])
    
    if idx == 4:
        ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)

plt.suptitle('Time-Domain Analysis: Individual Class Signals', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_03_Time_Domain_Stacked.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_03_Time_Domain_Stacked.png")

# ====================================================================================
# PLOT 4: FREQUENCY SPECTRUM (OVERLAPPED)
# ====================================================================================
print("Creating Plot 4: Frequency Spectrum (Overlapped)...")

fig, ax = plt.subplots(figsize=(14, 7))

for leak_type in leak_classes:
    for sample in plot_data[leak_type]:
        N = len(sample['signal'])
        yf = fft(sample['signal'])
        xf = fftfreq(N, 1/fs)[:N//2]
        power = 2.0/N * np.abs(yf[0:N//2])
        
        ax.plot(xf, power, color=class_colors[leak_type], alpha=0.4, linewidth=1.5)

# Create legend
legend_elements = [Line2D([0], [0], color=class_colors[cls], lw=3, label=cls) 
                   for cls in leak_classes]
ax.legend(handles=legend_elements, loc='upper right', framealpha=0.9, fontsize=11)

ax.set_xlabel('Frequency (Hz)', fontweight='bold', fontsize=14)
ax.set_ylabel('Magnitude', fontweight='bold', fontsize=14)
ax.set_title('Frequency Spectrum: All Leak Classes Comparison', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')
ax.set_xlim([0, fs/2])

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_04_Frequency_Spectrum_Overlapped.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_04_Frequency_Spectrum_Overlapped.png")

# ====================================================================================
# PLOT 5: FREQUENCY SPECTRUM (STACKED SUBPLOTS)
# ====================================================================================
print("Creating Plot 5: Frequency Spectrum (Stacked)...")

fig, axes = plt.subplots(5, 1, figsize=(14, 14))

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    for sample in plot_data[leak_type]:
        N = len(sample['signal'])
        yf = fft(sample['signal'])
        xf = fftfreq(N, 1/fs)[:N//2]
        power = 2.0/N * np.abs(yf[0:N//2])
        
        ax.plot(xf, power, color=class_colors[leak_type], alpha=0.6, linewidth=1.2)
    
    ax.set_ylabel('Magnitude', fontweight='bold', fontsize=12)
    ax.set_title(f'{leak_type}', fontsize=13, fontweight='bold',
                loc='left', color=class_colors[leak_type], pad=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xlim([0, fs/2])
    
    if idx == 4:
        ax.set_xlabel('Frequency (Hz)', fontweight='bold', fontsize=14)

plt.suptitle('Frequency Domain Analysis: Individual Class Spectra', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_05_Frequency_Spectrum_Stacked.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_05_Frequency_Spectrum_Stacked.png")

# ====================================================================================
# PLOT 6: SPECTROGRAMS (INDIVIDUAL FOR EACH CLASS)
# ====================================================================================
print("Creating Plot 6: Spectrograms...")

for leak_type in leak_classes:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    sample_signal = plot_data[leak_type][0]['signal']
    f, t, Sxx = signal.spectrogram(sample_signal, fs, nperseg=min(128, len(sample_signal)//4))
    
    pcm = ax.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), 
                       shading='gouraud', cmap='viridis')
    
    ax.set_ylabel('Frequency (Hz)', fontweight='bold', fontsize=14)
    ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)
    ax.set_title(f'Spectrogram: {leak_type}', fontsize=16, fontweight='bold', pad=20)
    
    cbar = plt.colorbar(pcm, ax=ax)
    cbar.set_label('Power Spectral Density (dB/Hz)', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    filename = leak_type.replace(' ', '_').replace('-', '_')
    plt.savefig(f'{output_dir}/Plot_06_Spectrogram_{filename}.png', bbox_inches='tight', dpi=300)
    plt.close()

print("  ✅ Saved: Plot_06_Spectrogram_*.png (5 files)")

# ====================================================================================
# PLOT 7: POWER SPECTRAL DENSITY (PSD) - WELCH METHOD
# ====================================================================================
print("Creating Plot 7: Power Spectral Density (Welch)...")

fig, ax = plt.subplots(figsize=(14, 7))

for leak_type in leak_classes:
    for sample in plot_data[leak_type]:
        freqs, psd = welch(sample['signal'], fs, nperseg=min(256, len(sample['signal'])//2))
        ax.semilogy(freqs, psd, color=class_colors[leak_type], alpha=0.4, linewidth=1.5)

legend_elements = [Line2D([0], [0], color=class_colors[cls], lw=3, label=cls) 
                   for cls in leak_classes]
ax.legend(handles=legend_elements, loc='upper right', framealpha=0.9, fontsize=11)

ax.set_xlabel('Frequency (Hz)', fontweight='bold', fontsize=14)
ax.set_ylabel('Power Spectral Density (V²/Hz)', fontweight='bold', fontsize=14)
ax.set_title('Power Spectral Density Analysis (Welch Method)', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--', which='both')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_07_Power_Spectral_Density.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_07_Power_Spectral_Density.png")

# ====================================================================================
# PLOT 8: AMPLITUDE DISTRIBUTION (HISTOGRAMS)
# ====================================================================================
print("Creating Plot 8: Amplitude Distributions...")

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    for sample in plot_data[leak_type]:
        ax.hist(sample['signal'], bins=40, color=class_colors[leak_type], 
                alpha=0.5, edgecolor='black', linewidth=0.5, density=True)
    
    ax.set_xlabel('Normalized Amplitude', fontweight='bold', fontsize=12)
    ax.set_ylabel('Probability Density', fontweight='bold', fontsize=12)
    ax.set_title(leak_type, fontsize=13, fontweight='bold', color=class_colors[leak_type])
    ax.grid(True, alpha=0.3, linestyle='--')

axes[5].axis('off')

plt.suptitle('Amplitude Distribution Analysis', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_08_Amplitude_Distributions.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_08_Amplitude_Distributions.png")

# ====================================================================================
# PLOT 9: BOX PLOTS - STATISTICAL METRICS
# ====================================================================================
print("Creating Plot 9: Statistical Metrics Box Plots...")

# Calculate statistics for all samples
stats_data = []
for leak_type in leak_classes:
    class_samples = X[y == leak_type]
    
    for sample in class_samples:
        stats_data.append({
            'Class': leak_type,
            'Mean': np.mean(sample),
            'Std': np.std(sample),
            'RMS': np.sqrt(np.mean(sample**2)),
            'Peak-to-Peak': np.ptp(sample),
            'Skewness': skew(sample),
            'Kurtosis': kurtosis(sample),
            'Energy': np.sum(sample**2),
            'Zero_Crossings': np.sum(np.diff(np.sign(sample)) != 0)
        })

stats_df = pd.DataFrame(stats_data)

metrics = ['Mean', 'Std', 'RMS', 'Peak-to-Peak', 'Skewness', 'Kurtosis']
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, metric in enumerate(metrics):
    ax = axes[idx // 3, idx % 3]
    
    bp = ax.boxplot([stats_df[stats_df['Class'] == cls][metric].values for cls in leak_classes],
                    labels=leak_classes,
                    patch_artist=True,
                    widths=0.6,
                    showfliers=True,
                    boxprops=dict(linewidth=1.5),
                    whiskerprops=dict(linewidth=1.5),
                    capprops=dict(linewidth=1.5),
                    medianprops=dict(linewidth=2, color='red'))
    
    for patch, cls in zip(bp['boxes'], leak_classes):
        patch.set_facecolor(class_colors[cls])
        patch.set_alpha(0.7)
    
    ax.set_ylabel(metric, fontweight='bold', fontsize=13)
    ax.set_xticklabels(leak_classes, rotation=30, ha='right', fontsize=10)
    ax.set_title(f'{metric} Distribution', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.suptitle('Statistical Metrics Comparison Across Classes', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_09_Statistical_BoxPlots.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_09_Statistical_BoxPlots.png")

# ====================================================================================
# PLOT 10: VIOLIN PLOTS - AMPLITUDE DISTRIBUTIONS
# ====================================================================================
print("Creating Plot 10: Violin Plots...")

fig, ax = plt.subplots(figsize=(14, 7))

# Prepare data for violin plot
violin_data = [stats_df[stats_df['Class'] == cls]['RMS'].values for cls in leak_classes]

parts = ax.violinplot(violin_data, positions=range(len(leak_classes)), 
                      showmeans=True, showmedians=True, widths=0.7)

for i, pc in enumerate(parts['bodies']):
    pc.set_facecolor(class_colors[leak_classes[i]])
    pc.set_alpha(0.7)

ax.set_xticks(range(len(leak_classes)))
ax.set_xticklabels(leak_classes, rotation=30, ha='right')
ax.set_ylabel('RMS Value', fontweight='bold', fontsize=14)
ax.set_xlabel('Leak Type', fontweight='bold', fontsize=14)
ax.set_title('RMS Distribution Across Leak Classes (Violin Plot)', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_10_Violin_RMS_Distribution.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_10_Violin_RMS_Distribution.png")

# ====================================================================================
# PLOT 11: AUTOCORRELATION FUNCTIONS
# ====================================================================================
print("Creating Plot 11: Autocorrelation Functions...")

fig, axes = plt.subplots(5, 1, figsize=(14, 14))

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    for sample in plot_data[leak_type]:
        sig = sample['signal']
        autocorr = np.correlate(sig - np.mean(sig), sig - np.mean(sig), mode='full')
        autocorr = autocorr[len(autocorr)//2:]
        autocorr = autocorr / autocorr[0]
        lags = np.arange(len(autocorr)) / fs
        
        ax.plot(lags[:min(500, len(lags))], autocorr[:min(500, len(autocorr))], 
                color=class_colors[leak_type], alpha=0.6, linewidth=1.5)
    
    ax.set_ylabel('Autocorrelation', fontweight='bold', fontsize=12)
    ax.set_title(f'{leak_type}', fontsize=13, fontweight='bold',
                loc='left', color=class_colors[leak_type], pad=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.axhline(y=0, color='black', linestyle='--', linewidth=1)
    
    if idx == 4:
        ax.set_xlabel('Lag (s)', fontweight='bold', fontsize=14)

plt.suptitle('Autocorrelation Function Analysis', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_11_Autocorrelation.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_11_Autocorrelation.png")

# ====================================================================================
# PLOT 12: ENVELOPE ANALYSIS (HILBERT TRANSFORM)
# ====================================================================================
print("Creating Plot 12: Envelope Analysis (Hilbert Transform)...")

fig, axes = plt.subplots(5, 1, figsize=(14, 14))

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    sample = plot_data[leak_type][0]['signal']
    time_axis = np.arange(len(sample)) / fs
    
    # Compute envelope using Hilbert transform
    analytic_signal = hilbert(sample)
    envelope = np.abs(analytic_signal)
    
    ax.plot(time_axis, sample, color=class_colors[leak_type], alpha=0.5, linewidth=1, label='Signal')
    ax.plot(time_axis, envelope, color='red', linewidth=2, label='Envelope')
    ax.plot(time_axis, -envelope, color='red', linewidth=2)
    
    ax.set_ylabel('Amplitude', fontweight='bold', fontsize=12)
    ax.set_title(f'{leak_type}', fontsize=13, fontweight='bold',
                loc='left', color=class_colors[leak_type], pad=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(loc='upper right', fontsize=9)
    
    if idx == 4:
        ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)

plt.suptitle('Signal Envelope Analysis (Hilbert Transform)', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_12_Envelope_Analysis.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_12_Envelope_Analysis.png")

# ====================================================================================
# PLOT 13: CUMULATIVE ENERGY
# ====================================================================================
print("Creating Plot 13: Cumulative Energy...")

fig, ax = plt.subplots(figsize=(14, 7))

for leak_type in leak_classes:
    for sample in plot_data[leak_type]:
        energy = np.cumsum(sample['signal']**2)
        energy_normalized = energy / energy[-1]
        time_axis = np.arange(len(sample['signal'])) / fs
        
        ax.plot(time_axis, energy_normalized, 
                color=class_colors[leak_type], alpha=0.4, linewidth=1.5)

legend_elements = [Line2D([0], [0], color=class_colors[cls], lw=3, label=cls) 
                   for cls in leak_classes]
ax.legend(handles=legend_elements, loc='lower right', framealpha=0.9, fontsize=11)

ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)
ax.set_ylabel('Normalized Cumulative Energy', fontweight='bold', fontsize=14)
ax.set_title('Cumulative Energy Distribution', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_13_Cumulative_Energy.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_13_Cumulative_Energy.png")

# ====================================================================================
# PLOT 14: CORRELATION MATRIX HEATMAP
# ====================================================================================
print("Creating Plot 14: Feature Correlation Heatmap...")

fig, ax = plt.subplots(figsize=(10, 8))

correlation_features = ['Mean', 'Std', 'RMS', 'Peak-to-Peak', 'Skewness', 'Kurtosis', 'Energy']
correlation_data = stats_df[correlation_features].corr()

im = ax.imshow(correlation_data, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)

ax.set_xticks(range(len(correlation_features)))
ax.set_yticks(range(len(correlation_features)))
ax.set_xticklabels(correlation_features, rotation=45, ha='right', fontsize=11)
ax.set_yticklabels(correlation_features, fontsize=11)

# Add correlation values
for i in range(len(correlation_features)):
    for j in range(len(correlation_features)):
        text = ax.text(j, i, f'{correlation_data.iloc[i, j]:.2f}',
                      ha="center", va="center", color="black", fontweight='bold', fontsize=10)

ax.set_title('Feature Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Correlation Coefficient', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_14_Correlation_Heatmap.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_14_Correlation_Heatmap.png")

# ====================================================================================
# PLOT 15: PCA VISUALIZATION (2D)
# ====================================================================================
print("Creating Plot 15: PCA 2D Visualization...")

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

fig, ax = plt.subplots(figsize=(12, 8))

for leak_type in leak_classes:
    mask = y == leak_type
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], 
              c=class_colors[leak_type], label=leak_type, 
              alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', 
             fontweight='bold', fontsize=14)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', 
             fontweight='bold', fontsize=14)
ax.set_title('Principal Component Analysis (PCA) - 2D Projection', 
            fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='best', framealpha=0.9, fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_15_PCA_2D.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_15_PCA_2D.png")

# ====================================================================================
# PLOT 16: t-SNE VISUALIZATION (2D)
# ====================================================================================
print("Creating Plot 16: t-SNE 2D Visualization...")

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X)

fig, ax = plt.subplots(figsize=(12, 8))

for leak_type in leak_classes:
    mask = y == leak_type
    ax.scatter(X_tsne[mask, 0], X_tsne[mask, 1], 
              c=class_colors[leak_type], label=leak_type, 
              alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

ax.set_xlabel('t-SNE Component 1', fontweight='bold', fontsize=14)
ax.set_ylabel('t-SNE Component 2', fontweight='bold', fontsize=14)
ax.set_title('t-SNE Visualization - 2D Projection', fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='best', framealpha=0.9, fontsize=11)
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_16_tSNE_2D.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_16_tSNE_2D.png")

# ====================================================================================
# PLOT 17: SIGNAL ENERGY DISTRIBUTION
# ====================================================================================
print("Creating Plot 17: Signal Energy Distribution...")

fig, ax = plt.subplots(figsize=(14, 7))

bp = ax.boxplot([stats_df[stats_df['Class'] == cls]['Energy'].values for cls in leak_classes],
                labels=leak_classes,
                patch_artist=True,
                widths=0.6,
                showfliers=True,
                boxprops=dict(linewidth=1.5),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5),
                medianprops=dict(linewidth=2, color='red'))

for patch, cls in zip(bp['boxes'], leak_classes):
    patch.set_facecolor(class_colors[cls])
    patch.set_alpha(0.7)

ax.set_ylabel('Signal Energy', fontweight='bold', fontsize=14)
ax.set_xlabel('Leak Type', fontweight='bold', fontsize=14)
ax.set_xticklabels(leak_classes, rotation=30, ha='right')
ax.set_title('Signal Energy Distribution Across Classes', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_17_Energy_Distribution.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_17_Energy_Distribution.png")

# ====================================================================================
# PLOT 18: ZERO CROSSING RATE
# ====================================================================================
print("Creating Plot 18: Zero Crossing Rate...")

fig, ax = plt.subplots(figsize=(14, 7))

bp = ax.boxplot([stats_df[stats_df['Class'] == cls]['Zero_Crossings'].values for cls in leak_classes],
                labels=leak_classes,
                patch_artist=True,
                widths=0.6,
                showfliers=True,
                boxprops=dict(linewidth=1.5),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5),
                medianprops=dict(linewidth=2, color='red'))

for patch, cls in zip(bp['boxes'], leak_classes):
    patch.set_facecolor(class_colors[cls])
    patch.set_alpha(0.7)

ax.set_ylabel('Zero Crossing Count', fontweight='bold', fontsize=14)
ax.set_xlabel('Leak Type', fontweight='bold', fontsize=14)
ax.set_xticklabels(leak_classes, rotation=30, ha='right')
ax.set_title('Zero Crossing Rate Comparison', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_18_Zero_Crossing_Rate.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_18_Zero_Crossing_Rate.png")

# ====================================================================================
# PLOT 19: PEAK DETECTION AND ANALYSIS
# ====================================================================================
print("Creating Plot 19: Peak Detection Analysis...")

fig, axes = plt.subplots(5, 1, figsize=(14, 14))

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    sample = plot_data[leak_type][0]['signal']
    time_axis = np.arange(len(sample)) / fs
    
    # Find peaks
    from scipy.signal import find_peaks
    peaks, properties = find_peaks(sample, height=0.5, distance=10)
    
    ax.plot(time_axis, sample, color=class_colors[leak_type], linewidth=1.5, label='Signal')
    ax.plot(time_axis[peaks], sample[peaks], 'rv', markersize=8, label=f'Peaks ({len(peaks)})')
    
    ax.set_ylabel('Amplitude', fontweight='bold', fontsize=12)
    ax.set_title(f'{leak_type} - {len(peaks)} peaks detected', fontsize=13, fontweight='bold',
                loc='left', color=class_colors[leak_type], pad=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(loc='upper right', fontsize=9)
    
    if idx == 4:
        ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)

plt.suptitle('Peak Detection and Analysis', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_19_Peak_Detection.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_19_Peak_Detection.png")

# ====================================================================================
# PLOT 20: WAVELET TRANSFORM (CONTINUOUS)
# ====================================================================================
print("Creating Plot 20: Continuous Wavelet Transform...")

import pywt

fig, axes = plt.subplots(5, 1, figsize=(14, 16))

for idx, leak_type in enumerate(leak_classes):
    ax = axes[idx]
    
    sample = plot_data[leak_type][0]['signal']
    
    # Perform continuous wavelet transform
    scales = np.arange(1, 128)
    coefficients, frequencies = pywt.cwt(sample, scales, 'morl', sampling_period=1/fs)
    
    pcm = ax.pcolormesh(np.arange(len(sample))/fs, frequencies, np.abs(coefficients), 
                       cmap='jet', shading='gouraud')
    
    ax.set_ylabel('Frequency (Hz)', fontweight='bold', fontsize=12)
    ax.set_title(f'{leak_type}', fontsize=13, fontweight='bold',
                loc='left', color=class_colors[leak_type], pad=10)
    
    cbar = plt.colorbar(pcm, ax=ax)
    cbar.set_label('Magnitude', fontsize=10)
    
    if idx == 4:
        ax.set_xlabel('Time (s)', fontweight='bold', fontsize=14)

plt.suptitle('Continuous Wavelet Transform Analysis', fontsize=18, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_20_Wavelet_Transform.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_20_Wavelet_Transform.png")

# ====================================================================================
# PLOT 21: CREST FACTOR ANALYSIS
# ====================================================================================
print("Creating Plot 21: Crest Factor Analysis...")

# Calculate crest factor for all samples
crest_factors = []
for leak_type in leak_classes:
    class_samples = X[y == leak_type]
    for sample in class_samples:
        peak = np.max(np.abs(sample))
        rms = np.sqrt(np.mean(sample**2))
        crest_factor = peak / (rms + 1e-8)
        crest_factors.append({'Class': leak_type, 'Crest_Factor': crest_factor})

cf_df = pd.DataFrame(crest_factors)

fig, ax = plt.subplots(figsize=(14, 7))

bp = ax.boxplot([cf_df[cf_df['Class'] == cls]['Crest_Factor'].values for cls in leak_classes],
                labels=leak_classes,
                patch_artist=True,
                widths=0.6,
                showfliers=True,
                boxprops=dict(linewidth=1.5),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5),
                medianprops=dict(linewidth=2, color='red'))

for patch, cls in zip(bp['boxes'], leak_classes):
    patch.set_facecolor(class_colors[cls])
    patch.set_alpha(0.7)

ax.set_ylabel('Crest Factor', fontweight='bold', fontsize=14)
ax.set_xlabel('Leak Type', fontweight='bold', fontsize=14)
ax.set_xticklabels(leak_classes, rotation=30, ha='right')
ax.set_title('Crest Factor Comparison (Peak/RMS Ratio)', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_21_Crest_Factor.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_21_Crest_Factor.png")

# ====================================================================================
# PLOT 22: FREQUENCY BAND ENERGY DISTRIBUTION
# ====================================================================================
print("Creating Plot 22: Frequency Band Energy Distribution...")

# Define frequency bands
freq_bands = {
    'Low (0-50 Hz)': (0, 50),
    'Medium (50-150 Hz)': (50, 150),
    'High (150-300 Hz)': (150, 300),
    'Very High (300-500 Hz)': (300, 500)
}

band_energy_data = []
for leak_type in leak_classes:
    for sample in plot_data[leak_type]:
        N = len(sample['signal'])
        yf = fft(sample['signal'])
        xf = fftfreq(N, 1/fs)[:N//2]
        power = 2.0/N * np.abs(yf[0:N//2])
        
        band_energies = {}
        for band_name, (f_low, f_high) in freq_bands.items():
            mask = (xf >= f_low) & (xf <= f_high)
            band_energies[band_name] = np.sum(power[mask])
        
        band_energies['Class'] = leak_type
        band_energy_data.append(band_energies)

band_df = pd.DataFrame(band_energy_data)

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, (band_name, (f_low, f_high)) in enumerate(freq_bands.items()):
    ax = axes[idx]
    
    bp = ax.boxplot([band_df[band_df['Class'] == cls][band_name].values for cls in leak_classes],
                    labels=leak_classes,
                    patch_artist=True,
                    widths=0.6,
                    showfliers=True,
                    boxprops=dict(linewidth=1.5),
                    whiskerprops=dict(linewidth=1.5),
                    capprops=dict(linewidth=1.5),
                    medianprops=dict(linewidth=2, color='red'))
    
    for patch, cls in zip(bp['boxes'], leak_classes):
        patch.set_facecolor(class_colors[cls])
        patch.set_alpha(0.7)
    
    ax.set_ylabel('Energy', fontweight='bold', fontsize=12)
    ax.set_xticklabels(leak_classes, rotation=30, ha='right', fontsize=10)
    ax.set_title(f'{band_name}', fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.suptitle('Frequency Band Energy Distribution', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_22_Frequency_Band_Energy.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_22_Frequency_Band_Energy.png")

# ====================================================================================
# PLOT 23: SAMPLE ENTROPY
# ====================================================================================
print("Creating Plot 23: Sample Entropy Analysis...")

def sample_entropy(signal, m=2, r=0.2):
    """Calculate sample entropy of a signal"""
    N = len(signal)
    std = np.std(signal)
    r = r * std
    
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
    
    def _phi(m):
        x = [[signal[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [len([1 for x_j in x if _maxdist(x_i, x_j) <= r]) / (N - m + 1.0) for x_i in x]
        return (N - m + 1.0)**(-1) * sum(np.log(C))
    
    return abs(_phi(m + 1) - _phi(m))

entropy_data = []
for leak_type in leak_classes:
    class_samples = X[y == leak_type]
    for sample in class_samples:
        try:
            ent = sample_entropy(sample)
            entropy_data.append({'Class': leak_type, 'Entropy': ent})
        except:
            pass

ent_df = pd.DataFrame(entropy_data)

fig, ax = plt.subplots(figsize=(14, 7))

bp = ax.boxplot([ent_df[ent_df['Class'] == cls]['Entropy'].values for cls in leak_classes],
                labels=leak_classes,
                patch_artist=True,
                widths=0.6,
                showfliers=True,
                boxprops=dict(linewidth=1.5),
                whiskerprops=dict(linewidth=1.5),
                capprops=dict(linewidth=1.5),
                medianprops=dict(linewidth=2, color='red'))

for patch, cls in zip(bp['boxes'], leak_classes):
    patch.set_facecolor(class_colors[cls])
    patch.set_alpha(0.7)

ax.set_ylabel('Sample Entropy', fontweight='bold', fontsize=14)
ax.set_xlabel('Leak Type', fontweight='bold', fontsize=14)
ax.set_xticklabels(leak_classes, rotation=30, ha='right')
ax.set_title('Signal Complexity Analysis (Sample Entropy)', fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='y', linestyle='--')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_23_Sample_Entropy.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_23_Sample_Entropy.png")

# ====================================================================================
# PLOT 24: INTER-CLASS DISTANCE MATRIX
# ====================================================================================
print("Creating Plot 24: Inter-Class Distance Matrix...")

# Calculate mean signal for each class
class_means = {}
for leak_type in leak_classes:
    class_samples = X[y == leak_type]
    class_means[leak_type] = np.mean(class_samples, axis=0)

# Calculate Euclidean distances between class means
distance_matrix = np.zeros((len(leak_classes), len(leak_classes)))
for i, cls1 in enumerate(leak_classes):
    for j, cls2 in enumerate(leak_classes):
        distance_matrix[i, j] = np.linalg.norm(class_means[cls1] - class_means[cls2])

fig, ax = plt.subplots(figsize=(10, 8))

im = ax.imshow(distance_matrix, cmap='YlOrRd', aspect='auto')

ax.set_xticks(range(len(leak_classes)))
ax.set_yticks(range(len(leak_classes)))
ax.set_xticklabels(leak_classes, rotation=45, ha='right', fontsize=11)
ax.set_yticklabels(leak_classes, fontsize=11)

# Add distance values
for i in range(len(leak_classes)):
    for j in range(len(leak_classes)):
        text = ax.text(j, i, f'{distance_matrix[i, j]:.1f}',
                      ha="center", va="center", color="black", fontweight='bold', fontsize=10)

ax.set_title('Inter-Class Distance Matrix (Euclidean)', fontsize=16, fontweight='bold', pad=20)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Euclidean Distance', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_24_Distance_Matrix.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_24_Distance_Matrix.png")

# ====================================================================================
# PLOT 25: RADAR CHART - CLASS COMPARISON
# ====================================================================================
print("Creating Plot 25: Radar Chart - Feature Comparison...")

# Calculate mean features for each class
radar_features = ['RMS', 'Peak-to-Peak', 'Std', 'Skewness', 'Kurtosis']
radar_data = {}

for leak_type in leak_classes:
    class_data = stats_df[stats_df['Class'] == leak_type]
    radar_data[leak_type] = [class_data[feat].mean() for feat in radar_features]

# Normalize data for radar chart
radar_data_norm = {}
for feat_idx in range(len(radar_features)):
    feat_values = [radar_data[cls][feat_idx] for cls in leak_classes]
    min_val, max_val = min(feat_values), max(feat_values)
    for cls in leak_classes:
        if cls not in radar_data_norm:
            radar_data_norm[cls] = []
        if max_val - min_val != 0:
            radar_data_norm[cls].append((radar_data[cls][feat_idx] - min_val) / (max_val - min_val))
        else:
            radar_data_norm[cls].append(0)

# Create radar chart
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = np.linspace(0, 2 * np.pi, len(radar_features), endpoint=False).tolist()
angles += angles[:1]

for leak_type in leak_classes:
    values = radar_data_norm[leak_type]
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=leak_type, color=class_colors[leak_type])
    ax.fill(angles, values, alpha=0.15, color=class_colors[leak_type])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(radar_features, fontsize=12, fontweight='bold')
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=10)
ax.grid(True, linestyle='--', alpha=0.5)
ax.set_title('Feature Comparison Across Classes (Normalized)', fontsize=16, fontweight='bold', pad=30)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)

plt.tight_layout()
plt.savefig(f'{output_dir}/Plot_25_Radar_Chart.png', bbox_inches='tight', dpi=300)
plt.close()
print("  ✅ Saved: Plot_25_Radar_Chart.png")

# ====================================================================================
# SAVE SUMMARY STATISTICS TO CSV
# ====================================================================================
print("\nSaving comprehensive statistics...")
stats_df.to_csv(f'{output_dir}/comprehensive_statistics.csv', index=False)
band_df.to_csv(f'{output_dir}/frequency_band_energy.csv', index=False)
cf_df.to_csv(f'{output_dir}/crest_factor_data.csv', index=False)

# Print summary
print(f"\n{'='*80}")
print("📊 STATISTICAL SUMMARY")
print(f"{'='*80}")
print("\nOverall Statistics by Class:")
print(stats_df.groupby('Class')[['Mean', 'Std', 'RMS', 'Peak-to-Peak']].describe())

print(f"\n{'='*80}")
print("✅ ALL PLOTS SUCCESSFULLY CREATED AND SAVED!")
print(f"{'='*80}")
print(f"\nOutput directory: {output_dir}/")
print("\nGenerated Plots:")
print("  01. Sample Distribution (Bar Chart)")
print("  02. Time-Domain Signals (Overlapped)")
print("  03. Time-Domain Signals (Stacked)")
print("  04. Frequency Spectrum (Overlapped)")
print("  05. Frequency Spectrum (Stacked)")
print("  06. Spectrograms (5 separate files)")
print("  07. Power Spectral Density")
print("  08. Amplitude Distributions")
print("  09. Statistical Box Plots")
print("  10. Violin Plots")
print("  11. Autocorrelation Functions")
print("  12. Envelope Analysis")
print("  13. Cumulative Energy")
print("  14. Correlation Heatmap")
print("  15. PCA 2D Visualization")
print("  16. t-SNE 2D Visualization")
print("  17. Energy Distribution")
print("  18. Zero Crossing Rate")
print("  19. Peak Detection")
print("  20. Wavelet Transform")
print("  21. Crest Factor")
print("  22. Frequency Band Energy")
print("  23. Sample Entropy")
print("  24. Inter-Class Distance Matrix")
print("  25. Radar Chart")
print(f"\n{'='*80}")
print("🎉 Complete! All publication-ready plots are available.")
print(f"{'='*80}\n")

LOADING AND PREPROCESSING DATA

📁 Processing Circumferential Crack: 8 files

📁 Processing Gasket Leak: 8 files

📁 Processing Longitudinal Crack: 8 files

📁 Processing No-leak: 8 files

📁 Processing Orifice Leak: 8 files

✅ Total processed samples: 40
✅ Signal length: 200
✅ Classes: ['Circumferential Crack', 'Gasket Leak', 'Longitudinal Crack', 'No-leak', 'Orifice Leak']

Creating Plot 1: Sample Count Distribution...


C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:141: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(leak_classes, rotation=30, ha='right')


  ✅ Saved: Plot_01_Sample_Distribution.png
Creating Plot 2: Time-Domain Signals (Overlapped)...
  ✅ Saved: Plot_02_Time_Domain_Overlapped.png
Creating Plot 3: Time-Domain Signals (Stacked)...
  ✅ Saved: Plot_03_Time_Domain_Stacked.png
Creating Plot 4: Frequency Spectrum (Overlapped)...
  ✅ Saved: Plot_04_Frequency_Spectrum_Overlapped.png
Creating Plot 5: Frequency Spectrum (Stacked)...
  ✅ Saved: Plot_05_Frequency_Spectrum_Stacked.png
Creating Plot 6: Spectrograms...
  ✅ Saved: Plot_06_Spectrogram_*.png (5 files)
Creating Plot 7: Power Spectral Density (Welch)...
  ✅ Saved: Plot_07_Power_Spectral_Density.png
Creating Plot 8: Amplitude Distributions...
  ✅ Saved: Plot_08_Amplitude_Distributions.png
Creating Plot 9: Statistical Metrics Box Plots...


C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:388: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([stats_df[stats_df['Class'] == cls][metric].values for cls in leak_classes],
C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:388: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([stats_df[stats_df['Class'] == cls][metric].values for cls in leak_classes],
C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:388: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([stats_df[stats_df['Class'] == cls][metric].values for cl

  ✅ Saved: Plot_09_Statistical_BoxPlots.png
Creating Plot 10: Violin Plots...
  ✅ Saved: Plot_10_Violin_RMS_Distribution.png
Creating Plot 11: Autocorrelation Functions...
  ✅ Saved: Plot_11_Autocorrelation.png
Creating Plot 12: Envelope Analysis (Hilbert Transform)...
  ✅ Saved: Plot_12_Envelope_Analysis.png
Creating Plot 13: Cumulative Energy...
  ✅ Saved: Plot_13_Cumulative_Energy.png
Creating Plot 14: Feature Correlation Heatmap...
  ✅ Saved: Plot_14_Correlation_Heatmap.png
Creating Plot 15: PCA 2D Visualization...
  ✅ Saved: Plot_15_PCA_2D.png
Creating Plot 16: t-SNE 2D Visualization...
  ✅ Saved: Plot_16_tSNE_2D.png
Creating Plot 17: Signal Energy Distribution...


...

  ✅ Saved: Plot_21_Crest_Factor.png
Creating Plot 22: Frequency Band Energy Distribution...


C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:849: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([band_df[band_df['Class'] == cls][band_name].values for cls in leak_classes],
C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:849: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([band_df[band_df['Class'] == cls][band_name].values for cls in leak_classes],
C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:849: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([band_df[band_df['Class'] == cls][band_name].values for

  ✅ Saved: Plot_22_Frequency_Band_Energy.png
Creating Plot 23: Sample Entropy Analysis...


C:\Users\Muhammad Umar\AppData\Local\Temp\ipykernel_29304\331946455.py:909: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot([ent_df[ent_df['Class'] == cls]['Entropy'].values for cls in leak_classes],


  ✅ Saved: Plot_23_Sample_Entropy.png
Creating Plot 24: Inter-Class Distance Matrix...
  ✅ Saved: Plot_24_Distance_Matrix.png
Creating Plot 25: Radar Chart - Feature Comparison...
  ✅ Saved: Plot_25_Radar_Chart.png

Saving comprehensive statistics...

📊 STATISTICAL SUMMARY

Overall Statistics by Class:
                       Mean                                                    \
                      count      mean       std       min       25%       50%   
Class                                                                           
Circumferential Crack   8.0  0.062120  0.387807 -0.622857 -0.088203  0.032244   
Gasket Leak             8.0  0.104579  0.192128 -0.174494  0.009317  0.055768   
Longitudinal Crack      8.0 -0.418477  0.798325 -1.734941 -0.618374 -0.086328   
No-leak                 8.0 -0.228731  0.681009 -1.829970 -0.294407  0.049285   
Orifice Leak            8.0 -0.156719  0.473777 -1.261034 -0.177370 -0.015872   

                                            Std